In [8]:
import torch
from transformers import BertModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification  # 추가된 임포트
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_upstage import UpstageEmbeddings
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
import sentencepiece as spm
from langchain_core.prompts import PromptTemplate
import time

In [9]:
# .env 파일에서 환경 변수 로드
load_dotenv()

True

In [10]:
# BERT 모델 및 토크나이저 로드
bert_tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
#bert_model = BertModel.from_pretrained("monologg/kobert")
bert_model = BertForSequenceClassification.from_pretrained("monologg/kobert", trust_remote_code=True)

/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-TNmHyL-r-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# ChatOpenAI 모델 초기화
gpt_llm = ChatOpenAI(
    model_name="gpt-4o-mini-2024-07-18",  # 또는 다른 OpenAI 모델
    temperature=0.2,
    openai_api_key=os.getenv("OPENAI_API_KEY")  # .env 파일에서 API 키 로드
)

In [5]:
# 임베딩 및 벡터스토어 설정
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = [
    "계약은 양 당사자 간의 합의에 의해 성립됩니다.",
    "계약의 위반 시 손해배상이 청구될 수 있습니다.",
    "계약서에 명시된 조항은 법적 구속력을 가집니다."
]
texts = []
for doc in documents:
    texts.extend(text_splitter.split_text(doc))
vectorstore = FAISS.from_texts(texts, passage_embeddings)

In [6]:
# 상태 정의
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage], operator.add]

In [12]:
# 노드 함수들
def evaluate_question(state):
    start_time = time.time()
    messages = state['messages']
    question = messages[-1].content
    inputs = bert_tokenizer(question, return_tensors='pt')
    outputs = bert_model(**inputs)
    classification = torch.argmax(outputs.logits, dim=1).item()
    classification_map = {0: "일반 법률 질문", 1: "복잡한 법률 질문"}
    evaluation = classification_map.get(classification, "알 수 없음")
    end_time = time.time()
    print(f"평가 단계 실행 시간: {end_time - start_time:.2f}초")
    return {"messages": [HumanMessage(content=f"평가 결과: {evaluation}")]}

# 테스트 코드
state = {'messages': [HumanMessage(content='계약 위반 시 손해배상 청구는 어떻게 하나요?')]}
evaluate_question(state)

평가 단계 실행 시간: 0.19초


{'messages': [HumanMessage(content='평가 결과: 일반 법률 질문')]}

In [13]:
def retrieve_info(state):
    start_time = time.time()
    messages = state['messages']
    query = messages[-1].content + " " + messages[0].content
    docs = vectorstore.similarity_search(query, k=3)
    retrieved_info = "\n".join([doc.page_content for doc in docs])
    end_time = time.time()
    print(f"검색 단계 실행 시간: {end_time - start_time:.2f}초")
    return {"messages": [HumanMessage(content=f"검색된 정보: {retrieved_info}")]}

# 테스트 코드
state = {'messages': [HumanMessage(content='계약 위반 시 손해배상 청구는 어떻게 하나요?'), HumanMessage(content='평가 결과: 일반 법률 질문')]}
retrieve_info(state)

검색 단계 실행 시간: 0.60초


{'messages': [HumanMessage(content='검색된 정보: 계약의 위반 시 손해배상이 청구될 수 있습니다.\n계약은 양 당사자 간의 합의에 의해 성립됩니다.\n계약서에 명시된 조항은 법적 구속력을 가집니다.')]}

In [14]:
def generate_response(state):
    start_time = time.time()
    messages = state['messages']
    evaluation = messages[1].content
    retrieved_info = messages[2].content
    user_input = messages[0].content
    
    prompt_template = PromptTemplate(
        input_variables=["evaluation", "retrieved_info", "user_input"],
        template="""
        당신은 전문 법률 AI 어시스턴트입니다.
        
        {evaluation}
        {retrieved_info}
        
        사용자 질문: {user_input}
        
        위의 평가 결과와 검색된 정보를 바탕으로 사용자에게 도움이 되는 법률 답변을 제공하세요.
        """
    )
    
    prompt = prompt_template.format(
        evaluation=evaluation,
        retrieved_info=retrieved_info,
        user_input=user_input
    )
    response = gpt_llm.invoke(prompt)
    end_time = time.time()
    print(f"생성 단계 실행 시간: {end_time - start_time:.2f}초")
    return {"messages": [HumanMessage(content=f"AI 응답: {response.content}")]}

# 테스트 코드
state = {'messages': [
    HumanMessage(content='계약 위반 시 손해배상 청구는 어떻게 하나요?'),
    HumanMessage(content='평가 결과: 일반 법률 질문'),
    HumanMessage(content='검색된 정보: 계약은 양 당사자 간의 합의에 의해 성립됩니다.')
]}
generate_response(state)

생성 단계 실행 시간: 4.93초


{'messages': [HumanMessage(content='AI 응답: 계약 위반 시 손해배상 청구를 진행하는 방법은 다음과 같습니다:\n\n1. **계약서 검토**: 우선, 계약서의 내용을 자세히 검토하여 위반된 조항이 무엇인지 확인합니다. 계약서에는 위반 시의 손해배상에 대한 조항이 포함되어 있을 수 있으므로 이를 참고하는 것이 중요합니다.\n\n2. **위반 사실 확인**: 상대방이 계약을 위반했음을 입증할 수 있는 증거를 수집합니다. 이메일, 문자 메시지, 계약서 사본 등 관련 자료를 확보하는 것이 좋습니다.\n\n3. **손해액 산정**: 계약 위반으로 인해 발생한 손해를 구체적으로 산정합니다. 손해는 직접적인 금전적 손실뿐만 아니라, 계약 이행으로 기대할 수 있었던 이익도 포함될 수 있습니다.\n\n4. **상대방에게 통지**: 계약 위반 사실과 손해배상 청구 의사를 상대방에게 공식적으로 통지합니다. 이때 내용증명 우편을 이용하면 증거로 활용할 수 있습니다.\n\n5. **협상 시도**: 상대방과의 협상을 통해 손해배상에 대한 합의를 시도할 수 있습니다. 이 과정에서 법률 전문가의 도움을 받는 것도 유익합니다.\n\n6. **소송 제기**: 협상이 실패할 경우, 법원에 손해배상 청구 소송을 제기할 수 있습니다. 이때는 법률 전문가의 조력을 받는 것이 좋습니다. 소송을 통해 법원의 판결을 받아 손해배상을 청구할 수 있습니다.\n\n7. **소송 절차**: 소송이 진행되면, 법원에서의 절차에 따라 증거 제출, 변론 등을 진행하게 됩니다. 최종적으로 법원에서 판결이 내려지면, 상대방에게 손해배상을 청구할 수 있습니다.\n\n계약 위반에 대한 손해배상 청구는 복잡할 수 있으므로, 법률 전문가와 상담하여 구체적인 상황에 맞는 조언을 받는 것이 중요합니다.')]}

In [15]:
# 그래프 생성
def create_graph():
    workflow = StateGraph(AgentState)
    
    workflow.add_node("평가", evaluate_question)
    workflow.add_node("검색", retrieve_info)
    workflow.add_node("생성", generate_response)
    
    workflow.set_entry_point("평가")
    workflow.add_edge("평가", "검색")
    workflow.add_edge("검색", "생성")
    workflow.add_edge("생성", END)
    
    return workflow.compile()

# 테스트 코드
create_graph()

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden'], 'metadata': {}, 'configurable': {}}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 0x16a9df0f0>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']), ChannelWrite<start:평가>(recurse=True, writes=[ChannelWriteEntry(channel='start:평가', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)]), '평가': PregelNode(config={'tags': [], 'metadata': {}, 'configurable': {}}, channels={'messages': 'messages'}, triggers=['start:평가'], mapper=functools.partial(<function _coerce_state at 0x17e5911c0>, <class '__main__.AgentState'>), writers=[ChannelWrite<평가,messages>(recurse=True, writes=[ChannelWriteEntry(channel='평가', value='평가', skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 0x16a9df0f

In [16]:
# 실행 함수
def run_pipeline(user_input):
    start_time = time.time()
    graph = create_graph()
    inputs = {"messages": [HumanMessage(content=user_input)]}
    for output in graph.stream(inputs):
        for key, value in output.items():
            if key == "생성":
                print(value['messages'][-1].content)
    end_time = time.time()
    print(f"전체 실행 시간: {end_time - start_time:.2f}초")

# 실행 예시
user_input = '계약 위반 시 손해배상 청구는 어떻게 하나요?'
run_pipeline(user_input)

평가 단계 실행 시간: 0.06초
검색 단계 실행 시간: 0.49초
생성 단계 실행 시간: 4.55초
AI 응답: 계약 위반 시 손해배상을 청구하는 과정은 다음과 같은 단계로 진행됩니다:

1. **계약서 검토**: 먼저, 해당 계약서를 자세히 검토하여 위반된 조항이 무엇인지 확인합니다. 계약서에 명시된 조항은 법적 구속력을 가지므로, 위반 사항이 계약서에 명시되어 있어야 합니다.

2. **손해 발생 확인**: 계약 위반으로 인해 발생한 손해를 확인합니다. 손해는 직접적인 금전적 손실뿐만 아니라, 계약 이행으로 인해 기대할 수 있었던 이익의 상실 등도 포함될 수 있습니다.

3. **상대방에게 통지**: 계약 위반 사실을 상대방에게 통지합니다. 이때, 위반 사항과 손해를 구체적으로 설명하고, 손해배상을 요구하는 내용을 포함해야 합니다. 통지서는 서면으로 작성하는 것이 좋습니다.

4. **협상 시도**: 상대방과의 협상을 통해 손해배상에 대한 합의를 시도할 수 있습니다. 이 과정에서 상대방이 손해배상에 응할 경우, 합의서를 작성하여 양 당사자가 서명하는 것이 중요합니다.

5. **법적 절차 진행**: 만약 협상이 실패할 경우, 법적 절차를 진행할 수 있습니다. 이 경우, 손해배상 청구 소송을 제기해야 하며, 이를 위해 변호사의 도움을 받는 것이 좋습니다. 소송을 제기할 때는 계약서, 손해 발생에 대한 증거, 통지서 등의 자료를 준비해야 합니다.

6. **법원 판결**: 법원에서 사건을 심리한 후 판결이 내려지면, 그에 따라 손해배상을 받을 수 있습니다.

계약 위반에 대한 손해배상 청구는 복잡할 수 있으므로, 법률 전문가와 상담하는 것이 좋습니다. 이를 통해 보다 정확하고 효과적인 대응을 할 수 있습니다.
전체 실행 시간: 5.13초
